In [1]:
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate,UpSampling2D
from tensorflow.keras.layers import Add
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from PIL import Image
import re
import shutil
import cv2
from sklearn.metrics import confusion_matrix
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import imageio
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
ROOT="D:/yolov8/mri/lgg-mri-segmentation/kaggle_3m"

In [ ]:
images = []
labels = []

for root, dirs, files in os.walk(ROOT):
    for file in files:
        if "mask" not in file:
            image_path = os.path.join(root, file)
            mask_file = file.replace(".tif", "_mask.tif")
            mask_path = os.path.join(root, mask_file)

            if os.path.exists(mask_path):
                # Check if the mask file exists
                images.append(image_path)
                labels.append(mask_path)


In [ ]:
def load_image(path):
    try:
        image=cv2.imread(path)
        if image is None:
            print(f"Failed to load image from path: {path}")
            return None
        print(f"Image shape after loading: {image.shape}")
        image=cv2.resize(image,(256,256))
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = np.expand_dims(image, axis=0)

        return image/255.0
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

In [ ]:
def load_label(path):
    try :
        image=cv2.imread(path)
        if image is None:
            print(f"Failed to load image from path: {path}")
            return None
        print(f"Image shape after loading: {image.shape}")
        image=cv2.resize(image,(256,256))
        image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        image = image.astype(np.float32)
        return np.expand_dims(image/255.0,axis=-1)
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

In [ ]:
def xyxy(labels, zoom=5):
    x_min = []
    y_min = []
    x_max = []
    y_max = []
    
    for path in labels:
        mask = load_label(path)
        
        occurrence = np.where(mask == 1)
        
        if occurrence[0].size == 0:
            xmin = ymin =xmax =ymax =5
        else:
            ymin = np.sort(occurrence[0])[0]
            ymax = np.sort(occurrence[0])[-1]
            xmin = np.sort(occurrence[1])[0]
            xmax = np.sort(occurrence[1])[-1]

        x_min.append(xmin - zoom)
        y_min.append(ymin - zoom)
        x_max.append(xmax + zoom)
        y_max.append(ymax + zoom)
    
    return (x_min, y_min, x_max, y_max)


In [ ]:
xy_xy=xyxy(labels,3)

In [ ]:
def show_sample(cnt):

    pic1=load_image(images[cnt])[0]
    pic2=load_label(labels[cnt])
    mask=load_image(images[cnt])[0]
    if 0 <= cnt < len(xy_xy[0]):
        cv2.rectangle(mask, (xy_xy[0][cnt], xy_xy[1][cnt]), (xy_xy[2][cnt], xy_xy[3][cnt]), (0, 1, 0), 0)
    else:
        print(f"Index {cnt} is out of range for xy_xy.")

    fig = plt.figure(figsize=(7, 6))

    ax = fig.add_subplot(1, 3, 1)
    ax.set_title("Brain Image")
    plt.imshow(pic1)

    ax = fig.add_subplot(1, 3, 2)
    ax.set_title("The Mask")
    plt.imshow(pic2)

    ax = fig.add_subplot(1, 3, 3)
    ax.set_title("                     Image with Rectangle where The Affected Area")
    plt.imshow(mask)

In [ ]:
for i in range(0,100,10):
    show_sample(i)

In [ ]:
data=pd.DataFrame(xy_xy).T
data.columns=["xmin","ymin","xmax","ymax"]
data.insert(loc=0, column='image_path', value=images)
data

In [ ]:
data['x_center'] = (data['xmin'] + data['xmax'])/2
data['y_center'] = (data['ymin'] + data['ymax'])/2
data['w'] = data['xmax'] - data['xmin']
data['h'] = data['ymax'] - data['ymin']
data['classes'] = 0
data['x_center'] = data['x_center']/256
data['w'] = data['w']/256
data['y_center'] = data['y_center']/256
data['h'] = data['h']/256
data.rename(columns={'x_center':'x',"y_center":"y"}, inplace=True)
data.head()

In [ ]:
df=data[["image_path","x","y","w","h","classes"]]
df

In [ ]:
files = list(df.image_path.unique())
files_train, files_valid = train_test_split(files, test_size = 0.2)
IMAGES = "images/"
LABELS = "labels/"
os.makedirs(IMAGES+"train", exist_ok=True)
os.makedirs(LABELS+"train", exist_ok=True)
os.makedirs(IMAGES+"valid", exist_ok=True)
os.makedirs(LABELS+"valid", exist_ok=True)

In [ ]:
train_filename = set(files_train)
valid_filename = set(files_valid)
for file in files:
    if file in train_filename:
        shutil.copy(file, IMAGES+"train")
    elif file in valid_filename:
        shutil.copy(file, IMAGES+"valid")

In [ ]:
for _, row in df.iterrows():    
    image_file = row['image_path']
    path=row['image_path'].split("/")[3]
    if image_file in train_filename:
        annotation_file = os.path.join(LABELS) + "train/" + path.replace('.tif', '.txt')
    else:
        annotation_file = os.path.join(LABELS) + "valid/" + path.replace('.tif', '.txt')
        
    with open(annotation_file, 'a') as ann_file:
        ann_file.write(f" {row['classes']} {row['x']} {row['y']} {row['w']} {row['h']}\n")

In [ ]:
%%writefile dataset.yaml
# Path
train: D:/yolov8/images/train
val: D:/yolov8/images/valid

# Classes
nc: 1
names: ['Tumour']

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from IPython import display
display.clear_output()
 
import ultralytics
ultralytics.checks()

In [ ]:
model.train(data="dataset.yaml", epochs=10, batch=8,lr0=0.00001)

In [ ]:
Image.open("runs/detect/train/results.png")